In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import pymongo
from pymongo.write_concern import WriteConcern

In [3]:
####### Add src folder to the system path so it can call utils
# Get the directory of the current script
script_dir = os.path.dirname("C:\Users\joanp\Projects\ctgov_rag\notebooks\RAGAS.ipynb")
# Get the parent directory of the current script
parent_dir = os.path.dirname(script_dir)
# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils.utils import connect_to_mongoDB

NameError: name '__file__' is not defined

In [2]:
load_dotenv(".env")
MONGODB_USER = os.getenv("MONGODB_USER")
MONGODB_PWD = os.getenv("MONGODB_PWD")

os.environ["LANGCHAIN_PROJECT"] = "RAGAS"

client = connect_to_mongoDB(MONGODB_USER, MONGODB_PWD)
ctgov = client["ctGov"]
preprocessed = ctgov["preprocessed"]

c:\Users\joanp\Projects\ctgov_rag\.venv\Lib\site-packages\pymongo\uri_parser.py:321: UserWarning: The value of retrywrites must be 'true' or 'false'
  return get_validated_options(opts, warn)


Pinged your deployment. You successfully connected to MongoDB!


In [3]:
studies = []
for study in preprocessed.find({})[:10]:
    nctId = study.pop("_id")
    studies.append({nctId:study})

In [4]:
from src.utils.utils import get_clinical_trial_study

from langchain_text_splitters import RecursiveJsonSplitter
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

# Helps asyncio run within Jupyter
import nest_asyncio
nest_asyncio.apply()


splitter = RecursiveJsonSplitter(max_chunk_size=2500)
docs = splitter.create_documents(texts=studies)

for d in docs:
    d.metadata["filename"] = d.page_content[2:13]

generator_llm = Ollama(model="mistral")
critic_llm = Ollama(model="mistral")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

c:\Users\joanp\Projects\ctgov_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
testset = generator.generate_with_langchain_docs(docs, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

Generating:  40%|████      | 4/10 [1:27:30<1:38:32, 985.39s/it]  Failed to batch ingest runs: LangSmithError("Failed to post https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Read timed out. (read timeout=10.0)\n")


In [ ]:
testset.to_pandas().to_csv("./data/RAGA_testset.v2.csv")